In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='', 
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The last person to land on moon was ...")
print(response.content)

The last person to walk on the Moon was Eugene Cernan, who was part of the Apollo 17 mission in December 1972. He left the lunar surface on December 19, 1972.


In [3]:
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [24]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-ii-backend-itc/job/R-52621")

page_data = loader.load().pop().page_content
print(page_data)






















Software Engineer II - Backend , ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previo

In [26]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page__data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page__data':page_data})
print(res.content)

```json
{
  "role": "Software Engineer II - Backend",
  "experience": "3-6 years",
  "skills": [
    "Java",
    "Python",
    "Microservices",
    "RESTful APIs",
    "GraphQL",
    "AWS",
    "Azure",
    "GCP",
    "MySQL",
    "MongoDB",
    "DynamoDB",
    "JUnit",
    "Jest",
    "Mocha",
    "Git",
    "Agile methodologies"
  ],
  "description": "Design and develop high-performance backend systems that support Nike's digital and in-store platforms. Collaborate with cross-functional teams to build powerful backend solutions that support seamless consumer experiences."
}
```


In [27]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II - Backend',
 'experience': '3-6 years',
 'skills': ['Java',
  'Python',
  'Microservices',
  'RESTful APIs',
  'GraphQL',
  'AWS',
  'Azure',
  'GCP',
  'MySQL',
  'MongoDB',
  'DynamoDB',
  'JUnit',
  'Jest',
  'Mocha',
  'Git',
  'Agile methodologies'],
 'description': "Design and develop high-performance backend systems that support Nike's digital and in-store platforms. Collaborate with cross-functional teams to build powerful backend solutions that support seamless consumer experiences."}

In [36]:
import pandas as pd

df = pd.read_csv("C:/Users/asus/ai_my_Project_2/cv_links.csv", encoding='ISO-8859-1')

df

,CV_Link,Role,Skills Included
0,https:// Datascience_sidharth_cv1.pdf,Data Science,"Python, SQL, Numpy,Pandas"
1,https:// DataEngineering_sidharth_cv1.pdf,Data Engineering,"Python, SQL, Airflow, Spark"
2,https:// AI_ML_sidharth_cv1.pdf,AI_ML,"LangChain, LLMs, Python, Transformers,Tensorfl..."
3,https:// Data_Analytics_sidharth_cv1.pdf,Data Analytics,"Python, SQL, Power BI,Excel"
4,https:// Fullstack_sidharth_cv1.pdf,Full Stack,"HTML,CSS,Java,Python,AWS,MySQL,MongoDB"


In [80]:
import uuid
import chromadb

client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name="cv_selector")

# Only index once
if collection.count() == 0:
    for _, row in df.iterrows():
        collection.add(
    documents=[row["Skills Included"]],
    metadatas={
        "cv_link": row["CV_Link"],
        "role": row["Role"],
        "skills": row["Skills Included"]   # ✅ Rename key
    },
    ids=[str(uuid.uuid4())]
)

In [71]:
job=json_res

In [72]:
job['skills']

['Java',
 'Python',
 'Microservices',
 'RESTful APIs',
 'GraphQL',
 'AWS',
 'Azure',
 'GCP',
 'MySQL',
 'MongoDB',
 'DynamoDB',
 'JUnit',
 'Jest',
 'Mocha',
 'Git',
 'Agile methodologies']

In [81]:
links = collection.query(query_texts=job['skills'], n_results=1).get('metadatas', [])[0]
links

[{'skills': 'HTML,CSS,Java,Python,AWS,MySQL,MongoDB',
  'role': 'Full Stack',
  'cv_link': 'https://\x85Fullstack_sidharth_cv1.pdf'}]

In [82]:
best_cv = links

In [ ]:
##"The system doesn’t read the full CV files themselves. Instead, I created a structured metadata table for each version of my CV — including the role it targets and a list of key skills it covers. These metadata entries are stored in a vector database (ChromaDB), and when a job description is scraped, the system uses skill-based similarity search to find the most relevant CV metadata entry. Once found, it links the corresponding CV file for email generation and download.

In [88]:
# 7. Generate Email to HR using STRICT CV-only skills
prompt_email = PromptTemplate.from_template(
    """
    ### JOB ROLE CONTEXT (from scraped job description):
    {job_role}

    ### YOUR AVAILABLE SKILLS (from CV):
    {cv_skills}

    ### INSTRUCTION:
    You are Sidharth Pattali, applying for the role mentioned above.
    Write a professional cold email to HR expressing interest in this role, and explain how your exact skills (listed above) make you a strong fit.

    ❌ Do NOT add any skills or tools not listed above.
    ✅ Use only the provided skills to justify your fit.
    ✅ Mention that your CV is attached.
    Keep the tone formal and concise.

    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm

# 🚨 Use only CV metadata, NOT job['skills']
email_response = chain_email.invoke({
    "job_role": job['role'],
    "cv_skills": best_cv[0]["skills"] 
})

print("\n📧 Generated Email:\n")
print(email_response.content)



📧 Generated Email:

Subject: Application for Software Engineer II - Backend Role

Dear Hiring Manager,

I am writing to express my interest in the Software Engineer II - Backend position at your esteemed organization. With a strong foundation in programming languages such as Java and Python, I am confident in my ability to design and develop efficient backend systems.

My experience with databases, including MySQL and MongoDB, enables me to effectively manage and optimize data storage and retrieval. Additionally, my knowledge of HTML and CSS, although primarily frontend-focused, provides a comprehensive understanding of the full technology stack.

As a skilled engineer with experience in cloud platforms like AWS, I am well-versed in deploying and managing scalable backend applications. I am excited about the opportunity to leverage my technical expertise to contribute to the success of your team.

I have attached my CV, which provides a detailed account of my experience and skills. I 

# 